# Recomendation Netflix Films and TV Shows by It's Description
### By : Risdan Kristori
### Start : 16/11/2020
### End :16/11/2020

In [ ]:
# Import Data and Libraries

import pandas as pd
import numpy as np

In [ ]:
import string
from nltk.corpus import stopwords

In [ ]:
netflix = pd.read_csv('../input/netflix-shows/netflix_titles.csv')
netflix.head(2)

In [ ]:
# Function to collect importance words

def convert_to_list(sentence):
    # Check characters to see if they are in punctuation
    nopunc = [char for char in sentence if char not in string.punctuation]
    #print(nopunc)
    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    #print(nopunc)
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
# Check the fuction if work
convert_to_list('i am from Indonesia, i love pizza from Italia!')

In [ ]:
# Create new coloum consist of importance words in list

netflix['Description_Words'] = netflix['description'].apply(convert_to_list)

netflix['Description_Words']

In [ ]:
# Import text processing Library

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Create description matrix
CV_Matrix = CountVectorizer(analyzer=convert_to_list).fit_transform(netflix['description'])

In [ ]:
print(CV_Matrix.shape)
print(CV_Matrix.nnz)

In [ ]:
# Create TfIdf matrix from Description Matrix
TfIdf_Matrix = TfidfTransformer().fit_transform(CV_Matrix)

In [ ]:
# Import Linear Kernel to count cosine similarity
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
Desc_sim = linear_kernel(TfIdf_Matrix,TfIdf_Matrix)

In [ ]:
indices = pd.Series(netflix.index, index=netflix['title'].apply(lambda x: x.lower()))
indices.head(3)

In [ ]:
def netflix_recomendation(title):
    title = title.lower()
    idx = indices[title]
    
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(Desc_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:20]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return netflix['title'].iloc[movie_indices]

In [ ]:
netflix_recomendation('Transformers Prime')

In [ ]:
netflix_recomendation('The Matrix Reloaded')